## Similar Questions search


The idea is that similar questions will have similar answers. So once we can find similar questions then we can just rank there answers ?

Strategy :

1. Search "topics" in question index.

2. get q_n (some number which can yeild 1000 answers) similar questions

3. Now rank these 1000 using RoBERTa-base


In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re
from collections import defaultdict
import numpy as np
from scipy import spatial
import numpy as np
import warnings
warnings.simplefilter(action='ignore')

In [17]:
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('../englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query

def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)
queries = []
#"In this example, the title and the question body of topic with id A.1 is printed."
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Sample Topics/Task1_Samples_V2.0.xml"
topic_reader = TopicReader(topic_file_path)
dict_q_a = defaultdict(list)
dict_aid_body = {}

for topic_id in ['A.31','A.78', 'A.101']:
#     topic_id = "A.31"
    title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
    body = topic_reader.get_topic(topic_id).question
    body_pro = re.sub('<[^<]+?>', '', body)
    query = title + '. ' + body_pro
    queries.append(query)
    query = query.lower()
    query = remove_stop(query)
    
#     print(topic_reader.get_topic(topic_id).lst_tags)
    print(topic_id, query)

    body = {
        "size": 1000,
         "query": {
               "more_like_this" : {
            "fields" : ["body"],
            "like" : query,
        }
#         "query": {
#                "more_like_this" : {
#             "fields" : ["title", "body"],
#             "like" : query,
#         }
#             "multi_match": {
#                             "query": query,
#                             "fields":  ["body","title"],
#                             "type": "cross_fields"
#                         }
#             "match": {
#                 "body": query
#             }
         
        }
    }

    res = es.search(index="arq_ans_ques", body=body)
    for result in res['hits']['hits']:
        dict_q_a[topic_id].append(result['_source']['post_id'])
        dict_aid_body[result['_source']['post_id']] = result['_source']['body']

A.31 doubt implication logical reasoning. evident truth table $p \to q$   $p$ false $q$ true, $p \to q$ true.  instances convince truth value.  example:  $p$: quadrilateral cyclic  $q$: opposite angles supplementary  $p$ false $q$ true, $p \to q$ true? 
A.78 $\sum_{n=1}^{\infty} a_n$ converges absolutely prove $\sum_{n=1}^{\infty} a_n^2$ converges. studying real analysis.i proof proofs exist. but, wrong proof.thanks advance     1.we show satisfies cauchy criterion series  2.consider arbitrary $\epsilon &gt; 0$  3.since $\sum a_n$ converges absolutely  exists s.t forall $n &gt; \geq n$ s.t $|\sum_{k=m+1}^{n} |a_k|| = \sum_{k=m+1}^{n} |a_k| &lt; \sqrt{\epsilon}$  4.we show suffices  5.consider arbitrary $n &gt; \geq n$  $|\sum_{k=m+1}^{n} a_k^2|  =  \sum_{k=m+1}^{n} a_k^2 \leq  (\sum_{k=m+1}^{n} |a_k|)^2 &lt; \sqrt{\epsilon}^2 = \epsilon$  6.so conclude $\sum a_n^2$ converges 
A.101 prove squared equal 2 induction.. asked prove induction $n^2\leq 2^n$, told true $ \forall n\in \mathbb{n}

In [3]:
# ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'

# dict_ques = {}

# with jsonlines.open(os.path.join(ques_path)) as reader:
#         for obj in tqdm(reader):
#             dict_ques[obj['post_id']] = obj
            
# ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
      
# dict_ans = {}
    
# with jsonlines.open(os.path.join(ans_path)) as reader:
#         for obj in tqdm(reader):
#             dict_ans[obj['post_id']] = obj

# dict_aid_body = {}

# for a_id in tqdm(list(dict_ans.keys())):
#     ans_body = re.sub('<[^<]+?>', '',  dict_ans[a_id]['body'])
#     qid = dict_ans[a_id]['parent_id']
#     ques_body = re.sub('<[^<]+?>', '',  dict_ques[qid]['body'])
#     ques_title = re.sub('<[^<]+?>', '',  dict_ques[qid]['title'])
#     dict_aid_body[a_id] = ques_title + '. ' + ques_body + '. ' + ans_body

In [4]:
from transformers import pipeline
from transformers import *
import torch
import gc

feat_ext = pipeline("feature-extraction", model="/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000", tokenizer='roberta-base', device=1) #our finetuned model

ques_emb = []
for query in tqdm(queries):
    ques_emb.append(np.mean(feat_ext(query)[0], axis=0))


del feat_ext
torch.cuda.empty_cache()
gc.collect()

430

In [5]:
def Sort_Tuple(tup):  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2])
    return tup

# os.environ["CUDA_VISIBLE_DEVICES"] = '2'

def reduce_str(tokenizer, body):
    encoded = tokenizer.encode(body)[:510]
    return tokenizer.decode(encoded)

In [6]:
dict_q_idx = {'A.31':0,'A.78':1, 'A.101':2}
result_list = []

feat_ext = pipeline("feature-extraction", model="/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000", tokenizer='roberta-base', device=1) #our finetuned model
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

count = 0
import math
for qid in ['A.31','A.78', 'A.101']:
# for qid in ['A.101']:
    tup_postid_sim = []
    for post_id in tqdm(dict_q_a[qid]):
        try:
            feat = feat_ext(reduce_str(tokenizer, dict_aid_body[int(post_id)]))[0]
            ans_emb = np.mean(feat, axis=0)
        except:
            print('CUDA error: an illegal memory access was encountered', post_id)
            break
        result = 1 - spatial.distance.cosine(ques_emb[dict_q_idx[qid]], ans_emb)
        
        if math.isnan(result):
            count+=1
            print(qid, post_id, count)
            break
        tup_postid_sim.append((qid,post_id,result))
    sorted_top = Sort_Tuple(tup_postid_sim[:500])[::-1]
    other_res = tup_postid_sim[500:]
    result_list.append(sorted_top+other_res)
#     result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:1000])

Token indices sequence length is longer than the specified maximum sequence length for this model (706 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1358 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (566 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (573 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (976 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (1172 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (641 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (839 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (760 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (887 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for th

Token indices sequence length is longer than the specified maximum sequence length for this model (665 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (544 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (598 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (911 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (570 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for thi

In [7]:
with open('es.mlt.final.top500sorted.dat', 'w') as eval_file:
    for res in result_list:
        count = 1
        for tuples in res:
            eval_file.write(tuples[0]+'\t'+ '1\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'rr'+'\n')
            count+=1

In [16]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m bpref /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv es.mlt.final.top500sorted.dat

bpref                 	A.101	0.3333
bpref                 	A.31	0.8711
bpref                 	A.78	0.2500
bpref                 	all	0.4848


In [15]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m bpref /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv ../runs/es.mlt.qai.merged3M.dat

bpref                 	A.101	0.2778
bpref                 	A.31	0.8400
bpref                 	A.78	0.2500
bpref                 	all	0.4559


In [8]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m ndcg /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv es.mlt.final.finetune.dat

ndcg                  	A.101	0.2160
ndcg                  	A.31	0.4261
ndcg                  	A.78	0.1054
ndcg                  	all	0.2492


In [10]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m ndcg /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv ../runs/es.mlt.qa-baseline.dat


ndcg                  	A.101	0.1891
ndcg                  	A.31	0.4209
ndcg                  	A.78	0.1799
ndcg                  	all	0.2633


In [11]:
from trectools import TrecRun, TrecEval, fusion

runs_path = '/data/szr207/projects/ArqMath/'
r1 = TrecRun("es.mlt.final.finetune.dat")
r2 = TrecRun("../runs/es.mlt.qa-baseline.dat")

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r2])

# Save run to disk with all its topics
fused_run.print_subset("rrf.final.finetune.dat", topics=fused_run.topics())

File rrf.final.finetune.dat writen.


In [12]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -q -m ndcg /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv rrf.final.finetune.dat

ndcg                  	A.101	0.2134
ndcg                  	A.31	0.4321
ndcg                  	A.78	0.1350
ndcg                  	all	0.2602


In [ ]:
result_list